# Messages in LangGraph: Communication Between Components

Messages in LangGraph are structured data objects that facilitate communication between different components of your graph, such as nodes, tools, and external systems. They provide a standardized way to pass information, maintain conversation history, and enable complex multi-turn interactions.

## Key Features:
- **Structured Communication**: Standardized format for data exchange
- **Conversation History**: Maintain context across multiple interactions
- **Type Safety**: Strongly typed message structures
- **Extensibility**: Support for custom message types and metadata

## Message Types:

### **1. Human Messages**
```python
from langchain_core.messages import HumanMessage

# Create a human message
human_msg = HumanMessage(content="Hello, how are you?")
print(human_msg.content)  # "Hello, how are you?"
print(human_msg.type)     # "human"
```

### **2. AI Messages**
```python
from langchain_core.messages import AIMessage

# Create an AI message
ai_msg = AIMessage(content="I'm doing well, thank you!")
print(ai_msg.content)  # "I'm doing well, thank you!"
print(ai_msg.type)     # "ai"
```

### **3. System Messages**
```python
from langchain_core.messages import SystemMessage

# Create a system message
system_msg = SystemMessage(content="You are a helpful assistant.")
print(system_msg.content)  # "You are a helpful assistant."
print(system_msg.type)     # "system"
```

### **4. Tool Messages**
```python
from langchain_core.messages import ToolMessage

# Create a tool message
tool_msg = ToolMessage(
    content="Weather data retrieved successfully",
    tool_call_id="weather_123"
)
print(tool_msg.content)        # "Weather data retrieved successfully"
print(tool_msg.tool_call_id)   # "weather_123"
```

## Message Properties:

### **Basic Properties**
```python
# All messages have these properties
message = HumanMessage(content="Hello")

print(message.content)    # Message content
print(message.type)       # Message type
print(message.id)        # Unique message ID
print(message.additional_kwargs)  # Additional metadata
```

### **Custom Properties**
```python
# Add custom properties to messages
message = HumanMessage(
    content="Hello",
    additional_kwargs={
        "user_id": "user_123",
        "session_id": "session_456",
        "timestamp": "2024-01-01T12:00:00Z"
    }
)

print(message.additional_kwargs["user_id"])  # "user_123"
```

## Message Lists and History:

### **Message List**
```python
from langchain_core.messages import HumanMessage, AIMessage

# Create a list of messages
messages = [
    HumanMessage(content="What's the weather like?"),
    AIMessage(content="I'll check the weather for you."),
    HumanMessage(content="Thank you!")
]

# Access messages
for msg in messages:
    print(f"{msg.type}: {msg.content}")
```

### **Message History Management**
```python
def add_message_to_history(history, new_message):
    """Add a new message to conversation history"""
    history.append(new_message)
    return history

def get_conversation_context(history, max_messages=10):
    """Get recent conversation context"""
    return history[-max_messages:] if len(history) > max_messages else history
```

## Message Processing:

### **Message Filtering**
```python
def filter_messages_by_type(messages, message_type):
    """Filter messages by type"""
    return [msg for msg in messages if msg.type == message_type]

# Get only human messages
human_messages = filter_messages_by_type(messages, "human")

# Get only AI messages
ai_messages = filter_messages_by_type(messages, "ai")
```

### **Message Transformation**
```python
def transform_messages(messages):
    """Transform messages for processing"""
    transformed = []
    for msg in messages:
        if msg.type == "human":
            # Add prefix to human messages
            transformed.append(HumanMessage(content=f"User: {msg.content}"))
        elif msg.type == "ai":
            # Add prefix to AI messages
            transformed.append(AIMessage(content=f"Assistant: {msg.content}"))
    return transformed
```

### **Message Validation**
```python
def validate_message(message):
    """Validate message structure"""
    if not message.content:
        return False, "Message content is empty"
    
    if not message.type:
        return False, "Message type is missing"
    
    if message.type not in ["human", "ai", "system", "tool"]:
        return False, f"Invalid message type: {message.type}"
    
    return True, "Message is valid"
```

## Advanced Message Patterns:

### **Message Threading**
```python
class MessageThread:
    def __init__(self, thread_id):
        self.thread_id = thread_id
        self.messages = []
    
    def add_message(self, message):
        """Add message to thread"""
        message.additional_kwargs["thread_id"] = self.thread_id
        self.messages.append(message)
    
    def get_thread_history(self):
        """Get complete thread history"""
        return self.messages
    
    def get_recent_messages(self, count=5):
        """Get recent messages from thread"""
        return self.messages[-count:] if len(self.messages) > count else self.messages
```

### **Message Routing**
```python
def route_message(message, routing_rules):
    """Route message based on content and rules"""
    content = message.content.lower()
    
    if "weather" in content:
        return "weather_handler"
    elif "news" in content:
        return "news_handler"
    elif "help" in content:
        return "help_handler"
    else:
        return "general_handler"
```

### **Message Aggregation**
```python
def aggregate_messages(messages, aggregation_type="recent"):
    """Aggregate messages for processing"""
    if aggregation_type == "recent":
        return messages[-5:]  # Last 5 messages
    elif aggregation_type == "summary":
        return create_message_summary(messages)
    elif aggregation_type == "context":
        return extract_context(messages)
    else:
        return messages

def create_message_summary(messages):
    """Create a summary of messages"""
    summary = f"Conversation with {len(messages)} messages:\n"
    for i, msg in enumerate(messages):
        summary += f"{i+1}. {msg.type}: {msg.content[:50]}...\n"
    return summary
```

## Message State Integration:

### **Messages in State**
```python
from typing import TypedDict, List
from langchain_core.messages import BaseMessage

class ChatState(TypedDict):
    messages: List[BaseMessage]
    user_id: str
    session_id: str
    context: dict

def process_messages(state: ChatState) -> dict:
    """Process messages in state"""
    messages = state["messages"]
    
    # Get last user message
    user_messages = [msg for msg in messages if msg.type == "human"]
    last_user_message = user_messages[-1] if user_messages else None
    
    # Process based on message content
    if last_user_message:
        response = generate_response(last_user_message.content)
        return {"messages": messages + [AIMessage(content=response)]}
    
    return {"messages": messages}
```

### **Message State Updates**
```python
def update_message_state(state: ChatState, new_message: BaseMessage) -> dict:
    """Update state with new message"""
    current_messages = state.get("messages", [])
    updated_messages = current_messages + [new_message]
    
    return {
        "messages": updated_messages,
        "message_count": len(updated_messages),
        "last_message_type": new_message.type,
        "last_message_time": datetime.now().isoformat()
    }
```

## Message Serialization:

### **Message to JSON**
```python
import json

def serialize_messages(messages):
    """Serialize messages to JSON"""
    serialized = []
    for msg in messages:
        serialized.append({
            "type": msg.type,
            "content": msg.content,
            "id": msg.id,
            "additional_kwargs": msg.additional_kwargs
        })
    return json.dumps(serialized)

def deserialize_messages(json_str):
    """Deserialize JSON to messages"""
    data = json.loads(json_str)
    messages = []
    for msg_data in data:
        if msg_data["type"] == "human":
            messages.append(HumanMessage(**msg_data))
        elif msg_data["type"] == "ai":
            messages.append(AIMessage(**msg_data))
        # ... handle other types
    return messages
```

### **Message Persistence**
```python
def save_messages_to_file(messages, filename):
    """Save messages to file"""
    with open(filename, 'w') as f:
        json.dump(serialize_messages(messages), f)

def load_messages_from_file(filename):
    """Load messages from file"""
    with open(filename, 'r') as f:
        return deserialize_messages(json.load(f))
```

## Best Practices:

### **1. Message Structure**
- Use appropriate message types for different purposes
- Include relevant metadata in additional_kwargs
- Keep message content focused and clear
- Use consistent message formatting

### **2. Message History**
- Implement proper message history management
- Use message filtering for specific use cases
- Consider message history limits for performance
- Implement message cleanup strategies

### **3. Message Processing**
- Validate messages before processing
- Handle message errors gracefully
- Implement message transformation consistently
- Use message routing for complex workflows

### **4. Performance**
- Limit message history size
- Use efficient message filtering
- Implement message caching when appropriate
- Monitor message processing performance

## Common Use Cases:

### **Chat Applications**
```python
# Multi-turn chat with message history
def chat_with_history(user_input, message_history):
    # Add user message
    message_history.append(HumanMessage(content=user_input))
    
    # Generate AI response
    ai_response = generate_response(message_history)
    message_history.append(AIMessage(content=ai_response))
    
    return message_history
```

### **Tool Integration**
```python
# Messages for tool calls
def call_tool_with_message(tool_name, parameters):
    tool_message = ToolMessage(
        content=f"Calling {tool_name} with {parameters}",
        tool_call_id=f"{tool_name}_{datetime.now().timestamp()}"
    )
    return tool_message
```

### **API Integration**
```python
# Messages for API communication
def create_api_message(endpoint, data):
    message = HumanMessage(
        content=json.dumps(data),
        additional_kwargs={
            "endpoint": endpoint,
            "timestamp": datetime.now().isoformat()
        }
    )
    return message
```

## Benefits:
- **Standardized Communication**: Consistent message format
- **Conversation Context**: Maintain interaction history
- **Type Safety**: Strongly typed message structures
- **Extensibility**: Support for custom message types
- **Integration**: Easy integration with external systems

## Tips for Success:
- Use appropriate message types for different scenarios
- Implement proper message history management
- Validate messages before processing
- Handle message errors gracefully
- Consider performance implications of message history
- Use message metadata for additional context
- Implement message cleanup strategies
- Test message processing thoroughly
